<a href="https://colab.research.google.com/github/murillo-borges/webscraping-livelo-esfera/blob/main/Webscraping_LiveloEsfera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# 1. Instalação de pacotes necessários
!pip install selenium
!apt-get update # para o ChromeDriver
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [26]:
# 2. Importações
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

In [27]:
# 3. Configurações do Chrome headless
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)

In [6]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

def extrair_livelo():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)

    try:
        url = "https://www.livelo.com.br/juntar-pontos/parceiros/amazon/AMZ"
        driver.get(url)

        # Espera até o benefício aparecer
        wait = WebDriverWait(driver, 15)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.css-1gdp5jf")))

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Benefício
        span = soup.find("span", class_="css-1gdp5jf")
        texto = span.get_text(strip=True)
        # Ex.: "Ganhe até 1 ponto a cada 1 real gasto"
        # Processar para formato "1x1 (Ganhe 1 ponto a cada real)"
        import re
        m = re.search(r"até\s*(\d+)\s*ponto[s]?\s*a\s*cada\s*(\d+)\s*real", texto, re.IGNORECASE)
        if m:
            pts = m.group(1)
            reais = m.group(2)
            beneficio_format = f"{pts}x{reais} ({texto.replace('até','').strip()})"
        else:
            # fallback
            beneficio_format = texto

        # Crédito
        div = soup.find("div", attrs={"data-testid":"Text_Typography"})
        credito_text = div.get_text(strip=True)

        resultado = f"""
📦 Benefício Amazon & Livelo = {beneficio_format}
💳 Creditado em até {credito_text}
""".strip()

    except Exception as e:
        resultado = f"❌ Erro ao extrair dados da Livelo: {str(e)}"
    finally:
        driver.quit()

    return resultado

In [7]:
livelo = extrair_livelo()
livelo

'📦 Benefício Amazon & Livelo = 1x1 (Ganhe  1 ponto  a cada 1 real gasto)\n💳 Creditado em até Ofertas'

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def extrair_esfera():
    options = Options()
    options.add_argument("--headless")  # Remova esta linha para debugar com navegador visível
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=options)

    try:
        url = "https://www.esfera.com.vc/p/amazon/e000100298"
        driver.get(url)

        # Espera o carregamento do título como indicativo de que o conteúdo principal foi carregado
        wait = WebDriverWait(driver, 15)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "product-title")))

        # Agora usa BeautifulSoup para extrair os campos HTML indicados
        soup = BeautifulSoup(driver.page_source, "html.parser")

        titulo = soup.find("h1", class_="product-title").text.strip()

        beneficio_section = soup.find("p", string="Beneficio")
        beneficio_div = beneficio_section.find_next("div")
        beneficio_raw = beneficio_div.contents[0].strip()
        beneficio_info = beneficio_div.find("p").text.strip()

        validade_section = soup.find("p", string="Validade")
        validade_text = validade_section.find_next("div").text.strip()

        credito_section = soup.find("p", string="Creditado em")
        credito_text = credito_section.find_next("div").text.strip()

        resultado = f"""
📦 Benefício {titulo} = {beneficio_raw} ({beneficio_info})
🗓️ Validade da promoção: Até {validade_text}
💳 Pontos creditados em até {credito_text}
Link de acesso: {url}
        """
    except Exception as e:
        resultado = f"❌ Erro ao extrair dados da Esfera: {str(e)}"
    finally:
        driver.quit()

    return resultado.strip()

In [5]:
esfera = extrair_esfera()
esfera

'📦 Benefício Amazon = 1x1 (Ganhe 1 ponto a cada real)\n🗓️ Validade da promoção: Até 31/12/25\n💳 Pontos creditados em até 90 dias do recebimento do produto\nLink de acesso: https://www.esfera.com.vc/p/amazon/e000100298'

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re

def extrair_resumo_amazon():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)

    try:
        # --- LIV ELO ---
        url_livelo = "https://www.livelo.com.br/juntar-pontos/parceiros/amazon/AMZ"
        driver.get(url_livelo)
        wait = WebDriverWait(driver, 15)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.css-1gdp5jf")))
        soup = BeautifulSoup(driver.page_source, "html.parser")

        texto_livelo = soup.find("span", class_="css-1gdp5jf").get_text(strip=True)
        m_livelo = re.search(r"até\s*(\d+)\s*ponto[s]?\s*a\s*cada\s*(\d+)\s*real", texto_livelo, re.IGNORECASE)
        if m_livelo:
            pts = m_livelo.group(1)
            reais = m_livelo.group(2)
            beneficio_livelo = f"{pts}x{reais} (Ganhe {pts} ponto{'s' if int(pts) > 1 else ''} a cada real)"
        else:
            beneficio_livelo = texto_livelo

        # --- ESFERA ---
        url_esfera = "https://www.esfera.com.vc/p/amazon/e000100298"
        driver.get(url_esfera)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "product-title")))
        soup = BeautifulSoup(driver.page_source, "html.parser")

        beneficio_section = soup.find("p", string="Beneficio")
        beneficio_div = beneficio_section.find_next("div")
        beneficio_raw = beneficio_div.contents[0].strip()
        m_esfera = re.match(r"(\d+)\s*x\s*(\d+)", beneficio_raw.lower())
        if m_esfera:
            pts = m_esfera.group(1)
            reais = m_esfera.group(2)
            beneficio_esfera = f"{pts}x{reais} (Ganhe {pts} ponto{'s' if int(pts) > 1 else ''} a cada real)"
        else:
            beneficio_info = beneficio_div.find("p").text.strip()
            beneficio_esfera = f"{beneficio_raw} ({beneficio_info})"

        # --- RESUMO FINAL ---
        resumo = f"""
Resumo do dia Amazon:

🟣 Livelo: {beneficio_livelo}
🔗 Link da promoção: {url_livelo}

🔴 Esfera: {beneficio_esfera}
🔗 Link da promoção: {url_esfera}
""".strip()

    except Exception as e:
        resumo = f"❌ Erro ao extrair dados: {str(e)}"
    finally:
        driver.quit()

    return resumo

In [12]:
resumo_amazon = extrair_resumo_amazon()
resumo_amazon

'Resumo do dia Amazon:\n\n🟣 Livelo: 1x1 (Ganhe 1 ponto a cada real)\n🔗 Link da promoção: https://www.livelo.com.br/juntar-pontos/parceiros/amazon/AMZ\n\n🔴 Esfera: 1x1 (Ganhe 1 ponto a cada real)\n🔗 Link da promoção: https://www.esfera.com.vc/p/amazon/e000100298'

📦 Funções de scrapping de dados


In [28]:
def extrair_livelo_amazon(driver):
    url = "https://www.livelo.com.br/juntar-pontos/parceiros/amazon/AMZ"
    driver.get(url)
    # ... extração do benefício da Amazon na Livelo ...
    return {
        "plataforma": "Livelo",
        "loja": "Amazon",
        "beneficio": "1x1 (Ganhe 1 ponto a cada real)",
        "link": url
    }

def extrair_esfera_amazon(driver):
    url = "https://www.esfera.com.vc/p/amazon/e000100298"
    driver.get(url)
    # ... extração do benefício da Amazon na Esfera ...
    return {
        "plataforma": "Esfera",
        "loja": "Amazon",
        "beneficio": "3x1 (Ganhe 3 pontos a cada real)",
        "link": url
    }

🔁 Juntando tudo dinamicamente:


In [29]:
def extrair_resumo_lojas():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)

    try:
        funcoes = [
            extrair_livelo_amazon,
            extrair_esfera_amazon,
            # No futuro: extrair_livelo_samsung, extrair_esfera_netshoes, etc.
        ]

        dados = []
        for func in funcoes:
            try:
                resultado = func(driver)
                dados.append(resultado)
            except Exception as e:
                dados.append({
                    "plataforma": func.__name__,
                    "loja": "Erro",
                    "beneficio": f"❌ Erro ao extrair: {str(e)}",
                    "link": "-"
                })

        # Agrupar por loja
        lojas = {}
        for item in dados:
            loja = item["loja"]
            if loja not in lojas:
                lojas[loja] = []
            lojas[loja].append(item)

        # Montar o texto final
        texto = ""
        for loja, registros in lojas.items():
            texto += f"\nResumo do dia {loja}:\n\n"
            for r in registros:
                emoji = "🟣" if r["plataforma"].lower() == "livelo" else "🔴"
                texto += f"{emoji} {r['plataforma']}: {r['beneficio']}\n🔗 Link da promoção: {r['link']}\n\n"

        return texto.strip()

    finally:
        driver.quit()

In [30]:
teste = extrair_resumo_lojas()
teste

'Resumo do dia Amazon:\n\n🟣 Livelo: 1x1 (Ganhe 1 ponto a cada real)\n🔗 Link da promoção: https://www.livelo.com.br/juntar-pontos/parceiros/amazon/AMZ\n\n🔴 Esfera: 3x1 (Ganhe 3 pontos a cada real)\n🔗 Link da promoção: https://www.esfera.com.vc/p/amazon/e000100298'

Enviar E-mail

In [33]:
#importações
from email.message import EmailMessage
import os
import smtplib

#definindo dados de envio do e-mail
remetente = os.environ['EMAIL_REMETENTE']
destinatario = os.environ['EMAIL_DESTINATARIO']
assunto = 'Seu resumo diário pontos Amazon'
mensagem = f"""Olá, aqui está o seu resumo dos pontos Amazon:

{teste}
"""

senha = os.environ['SENHA_GMAIL']

#crinado o e-mail
msg = EmailMessage()
msg['Subject'] = assunto
msg['From'] = remetente
msg['To'] = destinatario
msg.set_content(mensagem)

#enviando o e-mail
with smtplib.SMTP_SSL("smtp.gmail.com",465) as email:
  email.login(remetente,senha)
  email.send_message(msg)

print("O email foi enviado com sucesso!")

KeyError: 'EMAIL_REMETENTE'